In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax
#import pydot as pydot
from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

sub_sample = pd.read_csv('3rd_security_submit_sample.csv')

groupby_file_label = train.groupby(['file_id', 'label']).size().unstack().reset_index()

label = []
for i in range(groupby_file_label.shape[0]):
    label.append(list(pd.notnull(groupby_file_label.iloc[i,1:])).index(True))

test['file_id'] = test['file_id'] + 1000000

data = pd.concat([train.drop('label', axis = 1), test])

data['tid+index'] = data['tid'] + data['index']

gc.collect()

In [ ]:
def get_basic_feature(data):
    
    #样本的id
    feature = pd.DataFrame(data['file_id'].unique())
    feature.columns = ['file_id']
    
    ##########------------------------------------------------------------###################
    #每个fiel共出现过多少次
    temp = data.groupby(['file_id']).size().reset_index().rename(columns = {0: 'file_id_sum'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #每个fileid中index的最大值，最小值，中位数，均值，方差，skew
    temp = data.groupby(['file_id'])['index'].max().reset_index().rename(columns = {'index': 'index_max'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['index'].mean().reset_index().rename(columns = {'index': 'index_mean'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['index'].median().reset_index().rename(columns = {'index': 'index_median'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['index'].std().reset_index().rename(columns = {'index': 'index_std'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['index'].skew().reset_index().rename(columns = {'index': 'index_skew'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    
    print('file1 feature done')
    print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #每个fileid对应的tid的种类数，最大数量，最小数量，均值，方差
    temp_file_tid = data.groupby(['file_id', 'tid']).size().unstack().reset_index()
    temp_file_tid.columns = ['file_id'] + ['tid_' + str(x) for x in temp_file_tid.columns[1:]]
    #tid种类
    temp = pd.DataFrame(np.sum(pd.notnull(temp_file_tid.iloc[:,1:]), axis = 1)).rename(columns = {0: 'tid_unique_num'})
    temp['file_id'] = temp_file_tid['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #tid中的最大值
    temp = pd.DataFrame(temp_file_tid.iloc[:,1:].max(axis = 1)).rename(columns = {0: 'tid_max'})
    temp['file_id'] = temp_file_tid['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #tid中的最小值
    temp = pd.DataFrame(temp_file_tid.iloc[:,1:].min(axis = 1)).rename(columns = {0: 'tid_min'})
    temp['file_id'] = temp_file_tid['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #tid中的均值
    temp = pd.DataFrame(temp_file_tid.iloc[:,1:].mean(axis = 1)).rename(columns = {0: 'tid_mean'})
    temp['file_id'] = temp_file_tid['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #tid中的方差
    temp = pd.DataFrame(temp_file_tid.iloc[:,1:].std(axis = 1)).rename(columns = {0: 'tid_std'})
    temp['file_id'] = temp_file_tid['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    
#     #加入tfidf
#     temp_file_tid = temp_file_tid.drop('file_id', axis = 1).fillna(0)
#     transformer = TfidfTransformer(sublinear_tf=True, use_idf=True)
#     temp_file_tid = transformer.fit_transform(temp_file_tid)
#     temp_file_tid = pd.DataFrame(temp_file_tid.toarray())
#     temp_file_tid.columns = ['tfidf_file_tid_' + str(x) for x in temp_file_tid.columns]
#     temp_file_tid['file_id'] = feature['file_id']
#     feature = pd.merge(feature, temp_file_tid, how = 'left', on = 'file_id')
    
    #加入词频
    feature = pd.merge(feature, temp_file_tid, how = 'left', on = 'file_id')
    print('tid feature done')
    print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #每个fileid对应的api的种类数，最大数量，最小数量，均值，方差
    temp_file_api = data.groupby(['file_id', 'api']).size().unstack().reset_index()
    temp_file_api.columns = ['file_id'] + ['api_' + str(x) for x in temp_file_api.columns[1:]]
    #api种类
    temp = pd.DataFrame(np.sum(pd.notnull(temp_file_api.iloc[:,1:]), axis = 1)).rename(columns = {0: 'api_unique_num'})
    temp['file_id'] = temp_file_api['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #api的最大值
    temp = pd.DataFrame(temp_file_api.iloc[:,1:].max(axis = 1)).rename(columns = {0: 'api_max'})
    temp['file_id'] = temp_file_api['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #api中的最小值
    temp = pd.DataFrame(temp_file_api.iloc[:,1:].min(axis = 1)).rename(columns = {0: 'api_min'})
    temp['file_id'] = temp_file_api['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #api中的均值
    temp = pd.DataFrame(temp_file_api.iloc[:,1:].mean(axis = 1)).rename(columns = {0: 'api_mean'})
    temp['file_id'] = temp_file_api['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #api中的方差
    temp = pd.DataFrame(temp_file_api.iloc[:,1:].std(axis = 1)).rename(columns = {0: 'api_std'})
    temp['file_id'] = temp_file_api['file_id']
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    
#     #加入tfidf
#     temp_file_api = temp_file_api.drop('file_id', axis = 1).fillna(0)
#     transformer = TfidfTransformer(sublinear_tf=True, use_idf=True)
#     temp_file_api = transformer.fit_transform(temp_file_api)
#     temp_file_api = pd.DataFrame(temp_file_api.toarray())
#     temp_file_api.columns = ['tfidf_file_api_' + str(x) for x in temp_file_api.columns]
#     temp_file_api['file_id'] = feature['file_id']
#     feature = pd.merge(feature, temp_file_api, how = 'left', on = 'file_id')
    
    #加入词频
    feature = pd.merge(feature, temp_file_api, how = 'left', on = 'file_id')
    print('api feature done')
    print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #每个fileid对应的return value的最大值，最小值，均值，方差
    #方差
    temp = data.groupby(['file_id'])['return_value'].std().reset_index().rename(columns = {'return_value': 'return_value_std'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #最大值
    temp = data.groupby(['file_id'])['return_value'].max().reset_index().rename(columns = {'return_value': 'return_value_max'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #最小值
    temp = data.groupby(['file_id'])['return_value'].min().reset_index().rename(columns = {'return_value': 'return_value_min'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #均值
    temp = data.groupby(['file_id'])['return_value'].mean().reset_index().rename(columns = {'return_value': 'return_value_mean'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('return_value feature done')
    print(feature.shape)
    
    ##8.6 特征
    ##########------------------------------------------------------------###################
    #每个fileid对应的每个tid,return value的最大值
    temp_file_tid_value_mean = data.groupby(['file_id', 'tid'])['return_value'].mean().unstack().reset_index()
    temp_file_tid_value_mean.columns = ['file_id'] + ['file_tid_value_mean_' + str(x) for x in temp_file_tid_value_mean.columns[1:]]
    feature = pd.merge(feature, temp_file_tid_value_mean, how = 'left', on = 'file_id')
    print('file_tid_value_mean feature done')
    print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #每个fileid对应的每个api,return value的值
    #均值
    temp_file_api_value_mean = data.groupby(['file_id', 'api'])['return_value'].mean().unstack().reset_index()
    temp_file_api_value_mean.columns = ['file_id'] + ['temp_file_api_value_mean' + str(x) for x in temp_file_api_value_mean.columns[1:]]
    feature = pd.merge(feature, temp_file_api_value_mean, how = 'left', on = 'file_id')
    print('file_api_value_mean feature done')
    print(feature.shape)
    #方差
    temp_file_api_value_std = data.groupby(['file_id', 'api'])['return_value'].std().unstack().reset_index()
    temp_file_api_value_std.columns = ['file_id'] + ['temp_file_api_value_std' + str(x) for x in temp_file_api_value_std.columns[1:]]
    feature = pd.merge(feature, temp_file_api_value_std, how = 'left', on = 'file_id')
    print('file_api_value_std feature done')
    print(feature.shape)
    #最大值
    temp_file_api_value_max = data.groupby(['file_id', 'api'])['return_value'].max().unstack().reset_index()
    temp_file_api_value_max.columns = ['file_id'] + ['temp_file_api_value_max' + str(x) for x in temp_file_api_value_max.columns[1:]]
    feature = pd.merge(feature, temp_file_api_value_max, how = 'left', on = 'file_id')
    print('file_api_value_max feature done')
    print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #样本对应的api大概在index的哪个位置，最大位置，最小位置，四分位数，中位数，均值，
    def get_upper_quartile(x):
        return np.percentile(x, (25))

    def get_lower_quartile(x):
        return np.percentile(x, (75))
    #对应api在index的位置的均值
    temp = data.groupby(['file_id', 'api'])['index'].mean().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_api_index_mean_' + x for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_api_index_mean_ feature done')
    print(feature.shape)
    #对应api在index的位置的最大值
    temp = data.groupby(['file_id', 'api'])['index'].max().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_api_index_max_' + x for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_api_index_max_ feature done')
    print(feature.shape)
    #对应api在index的位置的最小值
    temp = data.groupby(['file_id', 'api'])['index'].min().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_api_index_min_' + x for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_api_index_min_ feature done')
    print(feature.shape)
    #对应api在index的位置的方差
    temp = data.groupby(['file_id', 'api'])['index'].std().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_api_index_std_' + x for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_api_index_std_ feature done')
    print(feature.shape)
    #对应api在index的位置的上四分位数
#     temp = data.groupby(['file_id', 'api'])['index'].max().unstack().reset_index()
#     temp.columns = ['file_id'] + ['file_id_api_index_max_' + x for x in temp.columns[1:]
#     feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    #对应api在index的位置的下四分位数
    
    ##########------------------------------------------------------------###################
    #样本对应的tid大概在index的哪个位置，最大位置，最小位置，四分位数，中位数，均值，
    #对应api在index的位置的均值
    temp = data.groupby(['file_id', 'tid'])['index'].mean().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_tid_index_mean_' + str(x) for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_tid_index_mean_ feature done')
    print(feature.shape)
    #对应api在index的位置的最大值
    temp = data.groupby(['file_id', 'tid'])['index'].max().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_tid_index_max_' + str(x) for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_tid_index_max_ feature done')
    print(feature.shape)
    #对应api在index的位置的最小值
    temp = data.groupby(['file_id', 'tid'])['index'].min().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_tid_index_min_' + str(x) for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id_tid_index_min_ feature done')
    print(feature.shape)
    #对应api在index的位置的方差
    temp = data.groupby(['file_id', 'tid'])['index'].std().unstack().reset_index()
    temp.columns = ['file_id'] + ['file_id_tid_index_std_' + str(x) for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')                            
    print('file_id_tid_index_std_ feature done')
    print(feature.shape)                              
     
        
    #8月7日特征
    ##########------------------------------------------------------------###################
    #重点api的单独统计量
    for api in ['NtWriteFile', 'LdrGetProcedureAddress', 'CopyFileA', 'NtOpenDirectoryObject', \
        'CopyFileA', 'closesocket', 'NtClose', 'InternetOpenA', 'LdrGetDllHandle', '__exception__']:
        X = data[data['api'] == api]
        temp = X.groupby(['file_id','tid']).size().unstack().reset_index()
        name = [api + '_file_id_tid_' + str(x) for x in temp.columns[1:]]
        temp.columns = ['file_id'] + name
        temp[api + '_file_id_tid_min'] = temp[name].min(axis = 1)
        temp[api + '_file_id_tid_max'] = temp[name].max(axis = 1)
        temp[api + '_file_id_tid_mean'] = temp[name].mean(axis = 1)
        temp[api + '_file_id_tid_std'] = temp[name].std(axis = 1)
        temp[api + '_file_id_tid_median'] = temp[name].median(axis = 1)

        feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
        print(api + '_file_id_tid_ done')
        print(feature.shape)
    
    ##########------------------------------------------------------------###################
    #每个fileid中tid+index的最大值，最小值，中位数，均值，方差，skew
    temp = data.groupby(['file_id'])['tid+index'].max().reset_index().rename(columns = {'tid+index': 'tid+index_max'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['tid+index'].mean().reset_index().rename(columns = {'tid+index': 'tid+index_mean'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['tid+index'].median().reset_index().rename(columns = {'tid+index': 'tid+index_median'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['tid+index'].std().reset_index().rename(columns = {'tid+index': 'tid+index_std'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    temp = data.groupby(['file_id'])['tid+index'].skew().reset_index().rename(columns = {'tid+index': 'tid+index_skew'})
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file2 feature done')
    print(feature.shape)
    
    temp = data.groupby(['file_id', 'index']).size().unstack().reset_index()
    temp.columns = ['file_id'] + ['index_' + str(x) for x in temp.columns[1:]]
    feature = pd.merge(feature, temp, how = 'left', on = 'file_id')
    print('file_id index feature done')
    print(feature.shape)
    #对temp_file_tid， temp_file_api， SVD分解特征？
    
    return feature

In [11]:
%%time
data_FE = get_basic_feature(data)

file1 feature done
(169717, 7)
tid feature done
(169717, 3740)
api feature done
(169717, 4056)
return_value feature done
(169717, 4060)
file_tid_value_mean feature done
(169717, 7788)
file_api_value_mean feature done
(169717, 8099)
file_api_value_std feature done
(169717, 8410)
file_api_value_max feature done
(169717, 8721)
file_id_api_index_mean_ feature done
(169717, 9032)
file_id_api_index_max_ feature done
(169717, 9343)
file_id_api_index_min_ feature done
(169717, 9654)
file_id_api_index_std_ feature done
(169717, 9965)
file_id_tid_index_mean_ feature done
(169717, 13693)
file_id_tid_index_max_ feature done
(169717, 17421)
file_id_tid_index_min_ feature done
(169717, 21149)
file_id_tid_index_std_ feature done
(169717, 24877)
NtWriteFile_file_id_tid_ done
(169717, 26368)
LdrGetProcedureAddress_file_id_tid_ done
(169717, 28699)
CopyFileA_file_id_tid_ done
(169717, 29072)
NtOpenDirectoryObject_file_id_tid_ done
(169717, 30855)
CopyFileA_file_id_tid_ done
(169717, 31228)
closesocket_f

In [5]:
data_FE

NameError: name 'data_FE' is not defined

In [12]:
train_FE = data_FE.iloc[:len(train['file_id'].unique()),:]

In [13]:
train_FE.shape

(116624, 43971)

In [14]:
test_FE = data_FE.iloc[len(train['file_id'].unique()):,:]

In [15]:
test_FE.shape

(53093, 43971)

In [16]:
del data_FE, train, test
gc.collect()

601

In [17]:
gc.collect()

0

In [19]:
label = pd.read_pickle('label.pkl')
label = label[0].values

In [20]:
train_FE['label'] = label

In [21]:
x0 = train_FE[train_FE['label'] == 0]

x1 = train_FE[train_FE['label'] != 0]

x0.shape, x1.shape

((111545, 43972), (5079, 43972))

In [22]:
x0_part = x0.copy()

In [23]:
x1.index = range(x1.shape[0])

In [24]:
x0_part.index = range(x0_part.shape[0])

In [25]:
permutation = np.random.permutation(x0_part.shape[0])

In [26]:
select = x0_part.iloc[permutation[:23000], :]

In [27]:
data_new = pd.concat([x1, select], axis = 0)

In [28]:
permutation = np.random.permutation(data_new.shape[0])

data_new = data_new.iloc[permutation, :]

label_new = data_new['label'].values

In [29]:
del x0_part
gc.collect()

23

In [30]:
dtrain = lgb.Dataset(data_new, label=label_new)

In [34]:
gc.collect()

103

In [32]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [33]:
hist = lgb.cv(params, dtrain, num_boost_round = 20000, verbose_eval=2, early_stopping_rounds=30, stratified=False)

[2]	cv_agg's multi_logloss: 1.53419 + 0.000219852
[4]	cv_agg's multi_logloss: 1.33802 + 0.000442797
[6]	cv_agg's multi_logloss: 1.17999 + 0.000682583
[8]	cv_agg's multi_logloss: 1.04865 + 0.000946963
[10]	cv_agg's multi_logloss: 0.937983 + 0.00115007
[12]	cv_agg's multi_logloss: 0.84245 + 0.00128171
[14]	cv_agg's multi_logloss: 0.759508 + 0.00147816
[16]	cv_agg's multi_logloss: 0.68626 + 0.00149649
[18]	cv_agg's multi_logloss: 0.622077 + 0.00161426
[20]	cv_agg's multi_logloss: 0.565737 + 0.00156486
[22]	cv_agg's multi_logloss: 0.515824 + 0.00158909
[24]	cv_agg's multi_logloss: 0.471132 + 0.0016304
[26]	cv_agg's multi_logloss: 0.431239 + 0.00165244
[28]	cv_agg's multi_logloss: 0.395558 + 0.00165973
[30]	cv_agg's multi_logloss: 0.363112 + 0.00168359
[32]	cv_agg's multi_logloss: 0.334167 + 0.00170116
[34]	cv_agg's multi_logloss: 0.308137 + 0.00176797
[36]	cv_agg's multi_logloss: 0.284646 + 0.00179845
[38]	cv_agg's multi_logloss: 0.263486 + 0.00183914
[40]	cv_agg's multi_logloss: 0.24431 +

KeyboardInterrupt: 

In [34]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [21]:
index = pd.read_csv('index.csv')

dtrain = lgb.Dataset(train_FE, label=label)

In [23]:
model = lgb.train(params=params, train_set=dtrain, num_boost_round=350, verbose_eval=10, valid_sets=dtrain)

KeyboardInterrupt: 

In [44]:
m = sorted(zip(dtrain.feature_name, model.feature_importance("gain")),key=lambda x: x[1], reverse=True)

In [64]:
t = [x for (x,y) in m if y>=100]
t

['file_id_api_index_min_NtWriteFile',
 'file_id_api_index_min_LdrGetProcedureAddress',
 'temp_file_api_value_maxNtProtectVirtualMemory',
 'file_id_api_index_min_CopyFileA',
 'file_id_api_index_max_CopyFileA',
 'file_id_api_index_min_NtOpenDirectoryObject',
 'file_id_api_index_std_closesocket',
 'file_id_api_index_min_NtClose',
 'index_std',
 'temp_file_api_value_meanNtClose',
 'file_id_api_index_mean_InternetOpenA',
 'file_id_api_index_min_LdrGetDllHandle',
 'file_id_api_index_mean_CopyFileA',
 'file_id_api_index_mean_NtOpenDirectoryObject',
 'file_id_api_index_mean___exception__',
 'file_id_api_index_min_InternetOpenUrlA',
 'file_id_api_index_min_GetSystemInfo',
 'temp_file_api_value_meanNtProtectVirtualMemory',
 'file_id_api_index_max_CreateServiceA',
 'index_5',
 'api_NtTerminateProcess',
 'api_min',
 'file_id_api_index_min_NtOpenSection',
 'index_max',
 'file_id_api_index_max_InternetOpenA',
 'index_2',
 'file_id_api_index_min_LdrLoadDll',
 'file_id_api_index_min_RegQueryValueExA',

In [29]:
len(t)

295

In [122]:
score = 1

In [223]:
def select(pred, num0, num1, num2, num3, num4, num5, selected_list, old_selected_data, 
           old_select_label):
    
    index_ = list(set(pred['file_id']) - set(selected_list))
    
    pred = pred.iloc[index_, :]
    
    #select num0 to num5
    new_list = []
    x0 = pred.sort_values('prob0', ascending=False)['file_id'].values[:num0]
    x1 = pred.sort_values('prob1', ascending=False)['file_id'].values[:num1]
    x2 = pred.sort_values('prob2', ascending=False)['file_id'].values[:num2]
    x3 = pred.sort_values('prob3', ascending=False)['file_id'].values[:num3]
    x4 = pred.sort_values('prob4', ascending=False)['file_id'].values[:num4]
    x5 = pred.sort_values('prob5', ascending=False)['file_id'].values[:num5]
    
    new_list.extend(x0)
    new_list.extend(x1)
    new_list.extend(x2)
    new_list.extend(x3)
    new_list.extend(x4)
    new_list.extend(x5)
    
    old_selected_data = pd.concat([old_selected_data, test_FE[t].iloc[new_list, :]])
    
    old_select_label.extend([0]*num0 + [1]*num1 + [2]*num2 + [3]*num3 + [4]*num4 + [5]*num5)
    
    selected_list.extend(new_list)
    
    return old_selected_data, old_select_label, selected_list, new_list

In [234]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [239]:
selected_list = []
old_selected_data = pd.DataFrame()
old_select_label = []
score = 1

In [240]:
gc.collect()

44

In [241]:
%%time
for n in range(10):
    print('*************第%s次 5cv*************'%n)
    logloss_list = []
    meta_test = pd.DataFrame()
    
    for i in range(5):
        gc.collect()
        print('*******%s*******'%i)
        idx_train = np.array(index[index['fold'] != i]['file_id'])
        idx_val = np.array(index[index['fold'] == i]['file_id'])


        X_train = train_FE.iloc[idx_train, :][t]
        y_train = pd.DataFrame(label)[0].values[idx_train]
        y_train = list(y_train)
        #加入半监督数据
        X_train = pd.concat([X_train, old_selected_data])
        print(old_selected_data.shape)
        print(X_train.shape)
        
        if len(old_select_label) != 0:
            y_train.extend(old_select_label)
            y_train = pd.DataFrame(y_train)[0].values
        else:
            y_train = pd.DataFrame(y_train)[0].values
        
        X_val = train_FE.iloc[idx_val, :][t]
        y_val = pd.DataFrame(label)[0].values[idx_val]
    
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val)
    
        
        model = lgb.train(params=params, train_set=dtrain, num_boost_round=1000, verbose_eval=50, 
                          valid_sets=dval, early_stopping_rounds=50)
        
        logloss_list.append(model.best_score['valid_0']['multi_logloss'])
        score_new = np.mean(logloss_list)
        print(np.mean(logloss_list))
        prob = model.predict(test_FE[t])

        meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

    if score_new<score:
        
        score = score_new
        
        print('start PL...')

        pred = pd.DataFrame(sub_sample.file_id)

        pred['prob0'] = meta_test[0].mean(axis = 1)
        pred['prob1'] = meta_test[1].mean(axis = 1)
        pred['prob2'] = meta_test[2].mean(axis = 1)
        pred['prob3'] = meta_test[3].mean(axis = 1)
        pred['prob4'] = meta_test[4].mean(axis = 1)
        pred['prob5'] = meta_test[5].mean(axis = 1)

        old_selected_data, old_select_label, selected_list, new_list= select(pred, 1000, 2, 2, 2, 1, 30, selected_list, old_selected_data, 
               old_select_label)
    else:
        
        print('End PL...')
        selected_list = list(set(selected_list) - set(new_list))

        break

*************第0次 5cv*************
*******0*******
(0, 0)
(93297, 295)
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 0.129425
[100]	valid_0's multi_logloss: 0.0283102
[150]	valid_0's multi_logloss: 0.0184063
[200]	valid_0's multi_logloss: 0.0167927
[250]	valid_0's multi_logloss: 0.0168028
Early stopping, best iteration is:
[226]	valid_0's multi_logloss: 0.0166624
0.0166624215279063
*******1*******
(0, 0)
(93297, 295)
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 0.127569
[100]	valid_0's multi_logloss: 0.0270006
[150]	valid_0's multi_logloss: 0.0171493
[200]	valid_0's multi_logloss: 0.0153622
[250]	valid_0's multi_logloss: 0.0151193
Early stopping, best iteration is:
[241]	valid_0's multi_logloss: 0.0150895
0.015875949137739917
*******2*******
(0, 0)
(93299, 295)
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 0.126757
[100]	valid_0's multi_logloss: 0.02

In [238]:
selected_list

[]

In [111]:
selected_list = []
old_selected_data = pd.DataFrame()
old_select_label = []

In [112]:
old_selected_data, old_select_label, selected_list, = select(pred, 10, 10, 10, 10, 10, 10, selected_list, old_selected_data, 
           old_select_label)

In [77]:
selected_list = [0, 2, 3]

In [79]:
selected_list.extend([0, 2, 3])

In [80]:
selected_list

[0, 2, 3, 0, 2, 3]

In [75]:
list(set(pred['file_id']) - set([0,1,2]))

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,


In [76]:
pred.iloc[list(set(pred['file_id']) - set([0,1,2])), :]

,file_id,prob0,prob1,prob2,prob3,prob4,prob5
3,3,0.999791,0.000011,0.000036,0.000021,0.000010,0.000131
4,4,0.999949,0.000005,0.000015,0.000006,0.000005,0.000020
5,5,0.999696,0.000012,0.000214,0.000013,0.000011,0.000054
6,6,0.999753,0.000019,0.000132,0.000023,0.000014,0.000059
7,7,0.999931,0.000008,0.000020,0.000012,0.000006,0.000022
8,8,0.998954,0.000052,0.000181,0.000110,0.000052,0.000651
9,9,0.994751,0.000389,0.003003,0.000695,0.000151,0.001011
10,10,0.999814,0.000010,0.000053,0.000011,0.000009,0.000103
11,11,0.999941,0.000006,0.000016,0.000006,0.000005,0.000026
12,12,0.999944,0.000005,0.000015,0.000006,0.000005,0.000025


In [128]:
100*pd.DataFrame(label)[0].value_counts()/111545

0    100.000000
5      3.045408
2      0.666995
3      0.536107
1      0.257295
4      0.047514
Name: 0, dtype: float64

In [57]:
pred.mean()

file_id    26546.000000
prob0          0.904965
prob1          0.007453
prob2          0.012395
prob3          0.012048
prob4          0.001061
prob5          0.062079
dtype: float64

In [58]:
pred.to_csv('/home/libo/Security/sub/8.12_5folds_cv_0.01472_mean_0.905.csv', index = None ,encoding = 'utf-8')

In [54]:
t = [x for (x,y) in m if y>=20]
len(t)

737

In [55]:
%%time
#logloss_list = []
meta_test2 = pd.DataFrame()
for i in range(5):
    gc.collect()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE.iloc[idx_train, :][t]
    y_train = pd.DataFrame(label)[0].values[idx_train]

    X_val = train_FE.iloc[idx_val, :][t]
    y_val = pd.DataFrame(label)[0].values[idx_val]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)
    
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=1000, verbose_eval=10, valid_sets=dval, early_stopping_rounds=100)
    

    prob = model.predict(test_FE[t])

    meta_test2 = pd.concat([meta_test2, pd.DataFrame(prob)], axis = 1)

*******0*******
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's multi_logloss: 0.905555
[20]	valid_0's multi_logloss: 0.525579
[30]	valid_0's multi_logloss: 0.319769
[40]	valid_0's multi_logloss: 0.200337
[50]	valid_0's multi_logloss: 0.12947
[60]	valid_0's multi_logloss: 0.0866278
[70]	valid_0's multi_logloss: 0.0605619
[80]	valid_0's multi_logloss: 0.044506
[90]	valid_0's multi_logloss: 0.0347382
[100]	valid_0's multi_logloss: 0.0286062
[110]	valid_0's multi_logloss: 0.0246737
[120]	valid_0's multi_logloss: 0.0221536
[130]	valid_0's multi_logloss: 0.0204297
[140]	valid_0's multi_logloss: 0.019267
[150]	valid_0's multi_logloss: 0.0185019
[160]	valid_0's multi_logloss: 0.0178676
[170]	valid_0's multi_logloss: 0.0174172
[180]	valid_0's multi_logloss: 0.0171368
[190]	valid_0's multi_logloss: 0.0169247
[200]	valid_0's multi_logloss: 0.0167395
[210]	valid_0's multi_logloss: 0.0166756
[220]	valid_0's multi_logloss: 0.0166408
[230]	valid_0's multi_logloss: 0.016

In [61]:
np.mean([0.0166202, 0.0158529, 0.0147042, 0.0140584, 0.0131922])

0.01488558

In [59]:
pred = pd.DataFrame(sub_sample.file_id)

pred['prob0'] = meta_test2[0].mean(axis = 1)
pred['prob1'] = meta_test2[1].mean(axis = 1)
pred['prob2'] = meta_test2[2].mean(axis = 1)
pred['prob3'] = meta_test2[3].mean(axis = 1)
pred['prob4'] = meta_test2[4].mean(axis = 1)
pred['prob5'] = meta_test2[5].mean(axis = 1)

In [60]:
pred.mean()

file_id    26546.000000
prob0          0.905353
prob1          0.007430
prob2          0.012233
prob3          0.012007
prob4          0.001072
prob5          0.061906
dtype: float64

In [62]:
pred.to_csv('/home/libo/Security/sub/8.12_5folds_cv_0.01488_mean_0.905.csv', index = None ,encoding = 'utf-8')

In [28]:
train_FE['file_id_api_index_max_CopyFileA'].fillna(0).value_counts()

0.0       113014
341.0        978
165.0        237
177.0         98
170.0         50
150.0         39
360.0         38
430.0         36
42.0          30
154.0         28
346.0         23
9.0           19
47.0          18
1056.0        17
181.0         15
173.0         13
58.0          13
343.0         12
11.0          11
6.0           11
28.0          11
791.0         11
19.0          11
2.0           10
162.0         10
4298.0         9
60.0           9
17.0           9
21.0           8
29.0           8
           ...  
2075.0         1
2044.0         1
2040.0         1
2024.0         1
2432.0         1
2440.0         1
2444.0         1
2455.0         1
2664.0         1
2657.0         1
2628.0         1
2626.0         1
2598.0         1
2587.0         1
2584.0         1
2581.0         1
2578.0         1
2577.0         1
2555.0         1
2554.0         1
2528.0         1
2527.0         1
2524.0         1
2510.0         1
2496.0         1
2491.0         1
2473.0         1
2462.0        

In [18]:
train_FE['file_id_sum'].value_counts()

5001      6698
44        3656
16        3011
10002     2298
43        1896
22        1828
20        1109
40         816
45         674
122        671
56         649
121        629
42         506
15003      471
20004      453
53         440
119        438
120        436
118        373
41         372
54         369
72         355
123        352
23         318
48         318
39         303
60         296
2          289
51         287
46         285
          ... 
12059        1
10010        1
24341        1
3807         1
20375        1
11196        1
9946         1
32465        1
216759       1
3743         1
7837         1
46984        1
14232        1
26770        1
4125         1
32911        1
37005        1
11132        1
4189         1
6236         1
8283         1
12313        1
16281        1
406399       1
26578        1
3999         1
35752        1
12187        1
10138        1
16680        1
Name: file_id_sum, dtype: int64

In [ ]:
train_FE['label'] = label

In [ ]:
X1 = train_FE[train_FE['label'] != 0]

In [ ]:
X2 = train_FE[train_FE['label'] == 0].iloc[:10000,:]

In [ ]:
X = pd.concat([X1, X2])
X.index = range(X.shape[0])

In [ ]:
index_ = list(X.index)

np.random.shuffle(index_)

X = X.iloc[index_,:]

In [ ]:
del X1, X2
gc.collect()

In [12]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [ ]:
X_temp = X.drop(['file_id', 'label'], axis = 1)
y_temp = X['label']

In [ ]:
del X
gc.collect()

In [31]:
dtrain = lgb.Dataset(train_FE ,label=label)

In [32]:
gc.collect()

0

In [34]:
model = lgb.train(params=params, train_set=dtrain, num_boost_round=350, verbose_eval=10, valid_sets=dtrain)

[10]	training's multi_logloss: 0.904326
[20]	training's multi_logloss: 0.523007
[30]	training's multi_logloss: 0.3162
[40]	training's multi_logloss: 0.196302
[50]	training's multi_logloss: 0.124758
[60]	training's multi_logloss: 0.0813246
[70]	training's multi_logloss: 0.0548878
[80]	training's multi_logloss: 0.0382917
[90]	training's multi_logloss: 0.0278159
[100]	training's multi_logloss: 0.0208051
[110]	training's multi_logloss: 0.0160793
[120]	training's multi_logloss: 0.0128055
[130]	training's multi_logloss: 0.010481
[140]	training's multi_logloss: 0.00879072
[150]	training's multi_logloss: 0.00748132
[160]	training's multi_logloss: 0.00643063
[170]	training's multi_logloss: 0.00561084
[180]	training's multi_logloss: 0.00495502
[190]	training's multi_logloss: 0.00440735
[200]	training's multi_logloss: 0.00394769
[210]	training's multi_logloss: 0.00356435
[220]	training's multi_logloss: 0.00323038
[230]	training's multi_logloss: 0.00293347
[240]	training's multi_logloss: 0.0026847

In [36]:
m = sorted(zip(dtrain.feature_name, model.feature_importance("gain")),key=lambda x: x[1], reverse=True)
#m

[('file_id_api_index_min_NtWriteFile', 37687.23753796704),
 ('file_id_api_index_min_LdrGetProcedureAddress', 30379.012817988172),
 ('temp_file_api_value_maxNtProtectVirtualMemory', 29477.747837364674),
 ('file_id_api_index_min_CopyFileA', 25366.41349761188),
 ('file_id_api_index_max_CopyFileA', 22671.389156520367),
 ('file_id_api_index_min_NtOpenDirectoryObject', 22512.10894084908),
 ('file_id_api_index_std_closesocket', 21530.414667367935),
 ('file_id_api_index_min_NtClose', 19403.537572093308),
 ('index_std', 14719.653366021812),
 ('temp_file_api_value_meanNtClose', 8458.2934554331),
 ('file_id_api_index_mean_InternetOpenA', 7128.786187805235),
 ('file_id_api_index_min_LdrGetDllHandle', 6472.062813986093),
 ('file_id_api_index_mean_CopyFileA', 6187.613959427923),
 ('file_id_api_index_mean_NtOpenDirectoryObject', 5725.847298458219),
 ('file_id_api_index_mean___exception__', 5500.183095788583),
 ('file_id_api_index_min_InternetOpenUrlA', 4572.485090538859),
 ('file_id_api_index_min_Get

In [37]:
t = [x for (x,y) in m if y>0]
print(len(t))
t

2493


['file_id_api_index_min_NtWriteFile',
 'file_id_api_index_min_LdrGetProcedureAddress',
 'temp_file_api_value_maxNtProtectVirtualMemory',
 'file_id_api_index_min_CopyFileA',
 'file_id_api_index_max_CopyFileA',
 'file_id_api_index_min_NtOpenDirectoryObject',
 'file_id_api_index_std_closesocket',
 'file_id_api_index_min_NtClose',
 'index_std',
 'temp_file_api_value_meanNtClose',
 'file_id_api_index_mean_InternetOpenA',
 'file_id_api_index_min_LdrGetDllHandle',
 'file_id_api_index_mean_CopyFileA',
 'file_id_api_index_mean_NtOpenDirectoryObject',
 'file_id_api_index_mean___exception__',
 'file_id_api_index_min_InternetOpenUrlA',
 'file_id_api_index_min_GetSystemInfo',
 'temp_file_api_value_meanNtProtectVirtualMemory',
 'file_id_api_index_max_CreateServiceA',
 'index_5',
 'api_NtTerminateProcess',
 'api_min',
 'file_id_api_index_min_NtOpenSection',
 'index_max',
 'file_id_api_index_max_InternetOpenA',
 'index_2',
 'file_id_api_index_min_LdrLoadDll',
 'file_id_api_index_min_RegQueryValueExA',

In [26]:
model

In [38]:
dtrain = lgb.Dataset(train_FE[t] ,label=label)

In [ ]:
hist = lgb.cv(params, dtrain, num_boost_round = 20000, verbose_eval=2, early_stopping_rounds=30, stratified=False)

[2]	cv_agg's multi_logloss: 1.52326 + 0.000331263
[4]	cv_agg's multi_logloss: 1.31905 + 0.000437269
[6]	cv_agg's multi_logloss: 1.15523 + 0.000337475
[8]	cv_agg's multi_logloss: 1.01948 + 0.000423183
[10]	cv_agg's multi_logloss: 0.905208 + 0.000468723
[12]	cv_agg's multi_logloss: 0.807024 + 0.00048664
[14]	cv_agg's multi_logloss: 0.722012 + 0.000520586
[16]	cv_agg's multi_logloss: 0.647843 + 0.00055794
[18]	cv_agg's multi_logloss: 0.582492 + 0.000634261
[20]	cv_agg's multi_logloss: 0.524698 + 0.000674342
[22]	cv_agg's multi_logloss: 0.473535 + 0.000720737
[24]	cv_agg's multi_logloss: 0.42803 + 0.000748263
[26]	cv_agg's multi_logloss: 0.387421 + 0.000764512
[28]	cv_agg's multi_logloss: 0.35103 + 0.000821207
[30]	cv_agg's multi_logloss: 0.31846 + 0.000867575
[32]	cv_agg's multi_logloss: 0.289191 + 0.00089614
[34]	cv_agg's multi_logloss: 0.262936 + 0.00094002
[36]	cv_agg's multi_logloss: 0.239476 + 0.000983964
[38]	cv_agg's multi_logloss: 0.218193 + 0.00100123
[40]	cv_agg's multi_logloss:

In [69]:
hist['multi_logloss-mean'][-1]

0.015879579470770806

In [39]:
hist = lgb.cv(params, dtrain, num_boost_round = 20000, verbose_eval=2, early_stopping_rounds=30, stratified=False)

[2]	cv_agg's multi_logloss: 1.52352 + 0.000138636
[4]	cv_agg's multi_logloss: 1.31929 + 0.000237412
[6]	cv_agg's multi_logloss: 1.15539 + 0.000391794
[8]	cv_agg's multi_logloss: 1.01992 + 0.00045307
[10]	cv_agg's multi_logloss: 0.905298 + 0.000474998
[12]	cv_agg's multi_logloss: 0.807007 + 0.00048989
[14]	cv_agg's multi_logloss: 0.72202 + 0.000471465
[16]	cv_agg's multi_logloss: 0.647684 + 0.000502803
[18]	cv_agg's multi_logloss: 0.582263 + 0.000553581
[20]	cv_agg's multi_logloss: 0.524549 + 0.000627712
[22]	cv_agg's multi_logloss: 0.473403 + 0.000668084
[24]	cv_agg's multi_logloss: 0.427977 + 0.000691804
[26]	cv_agg's multi_logloss: 0.387421 + 0.000695752
[28]	cv_agg's multi_logloss: 0.351143 + 0.00072256
[30]	cv_agg's multi_logloss: 0.318579 + 0.000801534
[32]	cv_agg's multi_logloss: 0.289351 + 0.000853513
[34]	cv_agg's multi_logloss: 0.263154 + 0.000915367
[36]	cv_agg's multi_logloss: 0.239583 + 0.000990543
[38]	cv_agg's multi_logloss: 0.218427 + 0.00106197
[40]	cv_agg's multi_loglo

In [43]:
1.1*len(hist['multi_logloss-mean'])

249.70000000000002

In [44]:
model = lgb.train(params=params, train_set=dtrain, num_boost_round=250, verbose_eval=10, valid_sets=dtrain)

[10]	training's multi_logloss: 0.904431
[20]	training's multi_logloss: 0.523047
[30]	training's multi_logloss: 0.316368
[40]	training's multi_logloss: 0.196539
[50]	training's multi_logloss: 0.125026
[60]	training's multi_logloss: 0.081487
[70]	training's multi_logloss: 0.0549064
[80]	training's multi_logloss: 0.0383199
[90]	training's multi_logloss: 0.0278035
[100]	training's multi_logloss: 0.02085
[110]	training's multi_logloss: 0.0160856
[120]	training's multi_logloss: 0.0128334
[130]	training's multi_logloss: 0.0105072
[140]	training's multi_logloss: 0.00881669
[150]	training's multi_logloss: 0.00749768
[160]	training's multi_logloss: 0.00646332
[170]	training's multi_logloss: 0.00564059
[180]	training's multi_logloss: 0.00496547
[190]	training's multi_logloss: 0.00441464
[200]	training's multi_logloss: 0.00395241
[210]	training's multi_logloss: 0.00356745
[220]	training's multi_logloss: 0.00323267
[230]	training's multi_logloss: 0.00294285
[240]	training's multi_logloss: 0.0026886

In [45]:
m = sorted(zip(dtrain.feature_name, model.feature_importance("gain")),key=lambda x: x[1], reverse=True)
#m

In [48]:
t2 = [x for (x,y) in m if y>200]
print(len(t2))
t2

172


['file_id_api_index_min_NtWriteFile',
 'file_id_api_index_min_LdrGetProcedureAddress',
 'file_id_api_index_min_CopyFileA',
 'file_id_api_index_min_NtOpenDirectoryObject',
 'file_id_api_index_max_CopyFileA',
 'file_id_api_index_std_closesocket',
 'file_id_api_index_min_NtClose',
 'temp_file_api_value_maxNtProtectVirtualMemory',
 'index_std',
 'temp_file_api_value_meanNtProtectVirtualMemory',
 'file_id_api_index_mean_InternetOpenA',
 'temp_file_api_value_meanNtClose',
 'file_id_api_index_min_LdrGetDllHandle',
 'file_id_api_index_mean___exception__',
 'file_id_api_index_mean_CopyFileA',
 'file_id_api_index_min_InternetOpenUrlA',
 'index_2',
 'file_id_api_index_min_GetSystemInfo',
 'index_5',
 'file_id_api_index_max_CreateServiceA',
 'api_min',
 'file_id_api_index_mean_NtOpenDirectoryObject',
 'file_id_api_index_min_NtOpenSection',
 'api_NtTerminateProcess',
 'index_max',
 'temp_file_api_value_meanNtMapViewOfSection',
 'file_id_api_index_min_LdrLoadDll',
 'file_id_api_index_max_LdrGetProce

In [47]:
pred = model.predict(test_FE[t])

In [49]:
pd.DataFrame(pred).mean()

0    0.905178
1    0.007487
2    0.012328
3    0.011952
4    0.001110
5    0.061945
dtype: float64

In [ ]:
groupby_file_label2 = groupby_file_label.copy()

groupby_file_label2[pd.notnull(groupby_file_label2)] = 1

groupby_file_label2 = groupby_file_label2.fillna(0)

In [ ]:
pred = pd.concat([pd.DataFrame(sub_sample['file_id']), pd.DataFrame(pred)], axis = 1)
pred.columns = ['file_id'] + ['prob' + str(x) for x in pred.columns[1:]]
pred.columns == sub_sample.columns

In [ ]:
pred.to_csv('/home/libo/Security/sub/cv_0.0150_mean_0.905.csv', index = None ,encoding = 'utf-8')

In [36]:
m2 = sorted(zip(dtrain.feature_name, model.feature_importance("gain")),key=lambda x: x[1], reverse=True)
m2

[('file_id_api_index_min_NtWriteFile', 43841.02045375109),
 ('file_id_api_index_min_LdrGetProcedureAddress', 30377.073818452656),
 ('file_id_api_index_min_CopyFileA', 24933.947728946805),
 ('file_id_api_index_max_CopyFileA', 23232.657127648592),
 ('temp_file_api_value_maxNtProtectVirtualMemory', 23114.648958861828),
 ('file_id_api_index_min_NtOpenDirectoryObject', 22369.728005364537),
 ('file_id_api_index_std_closesocket', 20837.157908678055),
 ('file_id_api_index_min_NtClose', 18074.0222876966),
 ('index_std', 13238.260853588581),
 ('file_id_api_index_mean_InternetOpenA', 9705.037433736026),
 ('temp_file_api_value_meanNtClose', 8697.545713841915),
 ('file_id_api_index_min_LdrGetDllHandle', 6797.13698752597),
 ('index_2', 6769.267821073532),
 ('file_id_api_index_max_NtOpenDirectoryObject', 5919.954669594765),
 ('temp_file_api_value_meanNtProtectVirtualMemory', 5243.7384214401245),
 ('file_id_api_index_max_CreateServiceA', 5202.178248018026),
 ('file_id_api_index_min_InternetOpenUrlA', 

In [43]:
t2 = [x for (x,y) in m2 if y>50]
print(len(t2))

428


In [44]:
print(1)

1


In [55]:
dtrain = lgb.Dataset(train_FE[t2] ,label=label)

In [46]:
hist = lgb.cv(params, dtrain, num_boost_round = 20000, verbose_eval=2, early_stopping_rounds=30, stratified=False)

[2]	cv_agg's multi_logloss: 1.52331 + 0.000254939
[4]	cv_agg's multi_logloss: 1.31915 + 0.000313912
[6]	cv_agg's multi_logloss: 1.1552 + 0.000300532
[8]	cv_agg's multi_logloss: 1.01951 + 0.00025818
[10]	cv_agg's multi_logloss: 0.905128 + 0.000387039
[12]	cv_agg's multi_logloss: 0.80688 + 0.000407773
[14]	cv_agg's multi_logloss: 0.721925 + 0.000435445
[16]	cv_agg's multi_logloss: 0.647586 + 0.000470625
[18]	cv_agg's multi_logloss: 0.582223 + 0.000469405
[20]	cv_agg's multi_logloss: 0.524465 + 0.000486511
[22]	cv_agg's multi_logloss: 0.473209 + 0.00052977
[24]	cv_agg's multi_logloss: 0.427702 + 0.000568502
[26]	cv_agg's multi_logloss: 0.387113 + 0.000606405
[28]	cv_agg's multi_logloss: 0.350795 + 0.000654784
[30]	cv_agg's multi_logloss: 0.318278 + 0.000726316
[32]	cv_agg's multi_logloss: 0.289014 + 0.0007887
[34]	cv_agg's multi_logloss: 0.262807 + 0.000843961
[36]	cv_agg's multi_logloss: 0.239224 + 0.000888168
[38]	cv_agg's multi_logloss: 0.217984 + 0.000948613
[40]	cv_agg's multi_loglos

KeyboardInterrupt: 

In [195]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [81]:
dtrain = lgb.Dataset(train_FE[t2] ,label=label)
model = lgb.train(params=params, train_set=dtrain, num_boost_round=int(1.1*238), verbose_eval=10, valid_sets=dtrain)

[10]	training's multi_logloss: 0.90396
[20]	training's multi_logloss: 0.522941
[30]	training's multi_logloss: 0.316348
[40]	training's multi_logloss: 0.196398
[50]	training's multi_logloss: 0.124934
[60]	training's multi_logloss: 0.0815143
[70]	training's multi_logloss: 0.055015
[80]	training's multi_logloss: 0.0384584
[90]	training's multi_logloss: 0.0279843
[100]	training's multi_logloss: 0.0210835
[110]	training's multi_logloss: 0.016425
[120]	training's multi_logloss: 0.0131859
[130]	training's multi_logloss: 0.0108641
[140]	training's multi_logloss: 0.00920504
[150]	training's multi_logloss: 0.00789662
[160]	training's multi_logloss: 0.00688678
[170]	training's multi_logloss: 0.00607822
[180]	training's multi_logloss: 0.00540943
[190]	training's multi_logloss: 0.00486166
[200]	training's multi_logloss: 0.00439971
[210]	training's multi_logloss: 0.00401478
[220]	training's multi_logloss: 0.003686
[230]	training's multi_logloss: 0.00339273
[240]	training's multi_logloss: 0.003138
[2

In [82]:
pred = model.predict(test_FE[t2])

In [83]:
pd.DataFrame(pred).mean()

0    0.905037
1    0.007475
2    0.012389
3    0.011929
4    0.001137
5    0.062032
dtype: float64

In [84]:
groupby_file_label2 = groupby_file_label.copy()
groupby_file_label2[pd.notnull(groupby_file_label2)] = 1
groupby_file_label2 = groupby_file_label2.fillna(0)

In [85]:
pred = pd.concat([pd.DataFrame(sub_sample['file_id']), pd.DataFrame(pred)], axis = 1)
pred.columns = ['file_id'] + ['prob' + str(x) for x in pred.columns[1:]]
pred.columns == sub_sample.columns

array([ True,  True,  True,  True,  True,  True,  True])

In [86]:
pred.to_csv('/home/libo/Security/sub/8.9_cv_0.01480_mean_0.905.csv', index = None ,encoding = 'utf-8')

In [87]:
kfold = pd.read_csv('index.csv')

In [88]:
pred = [] 

In [89]:
params = {"objective": "multiclass",
          "boosting_type": "gbdt",
          "learning_rate": 0.03,
          "num_leaves": 15,
          # "max_bin": 128,
          "feature_fraction": 0.85,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
          "subsample": 0.85,
          #'metric':'logloss',
#            'lambda_l1':0.1,
#            'lambda_l2':0.1,
          'seed':666,
          'num_class':6
          }

In [90]:
for i in range(5):
    print('********************第%s折*******************'%i)
    
    train_index = list(kfold[kfold['fold'] != i]['file_id'])
    test_index = list(kfold[kfold['fold'] == i]['file_id'])
    
    X_train = train_FE[t2].iloc[train_index,:]
    X_test = train_FE[t2].iloc[test_index,:]
    y_train = pd.DataFrame(label)[0][train_index].values
    y_test = pd.DataFrame(label)[0][test_index].values
    
    dtrain = lgb.Dataset(X_train ,label=y_train)
    dtest = lgb.Dataset(X_test ,label=y_test)
    
    model = lgb.train(params=params, train_set=dtrain, num_boost_round=10000, verbose_eval=10, valid_sets=dtest, 
                      early_stopping_rounds=100)
    
    p = model.predict(test_FE[t2], num_iteration=model.best_iteration)
    
    pred.append(p)

********************第0折*******************
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's multi_logloss: 1.16325
[20]	valid_0's multi_logloss: 0.814529
[30]	valid_0's multi_logloss: 0.588625
[40]	valid_0's multi_logloss: 0.433169
[50]	valid_0's multi_logloss: 0.322923
[60]	valid_0's multi_logloss: 0.243178
[70]	valid_0's multi_logloss: 0.184934
[80]	valid_0's multi_logloss: 0.142299
[90]	valid_0's multi_logloss: 0.110719
[100]	valid_0's multi_logloss: 0.0872533
[110]	valid_0's multi_logloss: 0.0698432
[120]	valid_0's multi_logloss: 0.0568477
[130]	valid_0's multi_logloss: 0.0471449
[140]	valid_0's multi_logloss: 0.0399365
[150]	valid_0's multi_logloss: 0.0345333
[160]	valid_0's multi_logloss: 0.0305167
[170]	valid_0's multi_logloss: 0.0273911
[180]	valid_0's multi_logloss: 0.0249569
[190]	valid_0's multi_logloss: 0.0231356
[200]	valid_0's multi_logloss: 0.0216918
[210]	valid_0's multi_logloss: 0.0205212
[220]	valid_0's multi_logloss: 0.0196629
[230]	valid

In [91]:
np.mean([0.0161496, 0.0150379, 0.0141679, 0.0135601, 0.0128535])

0.0143538

In [101]:
pred = pd.DataFrame(pred[0]) + pd.DataFrame(pred[1]) + pd.DataFrame(pred[2]) + pd.DataFrame(pred[3]) + pd.DataFrame(pred[4])

In [ ]:
pred/5

In [70]:
train_FE.shape

(116624, 43971)

In [71]:
test_FE.shape

(53093, 43971)

In [72]:
data = pd.concat([train_FE, test_FE])

In [74]:
data.drop('file_id', axis = 1, inplace = True)

KeyboardInterrupt: 

In [75]:
gc.collect()

609

In [278]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax
#import pydot as pydot
from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

In [266]:
gc.collect()

4522

In [267]:
temp_X = pd.concat([train_FE[t], test_FE[t]])

TypeError: cannot concatenate a non-NDFrame object

In [245]:
temp_X.fillna(-1, inplace = True)

In [12]:
def RankGauss(data):
    print('Start RankGauss...')
    for col in data.columns:
        #print('RankGauss...%s'%col)
        x = data[col].rank(method = 'dense')
        max_ = x.max()
        min_ = x.min()
        mean_ = (max_ + min_)/2
        
        x2 = (x - mean_).values/(0.0001+max_ - mean_)
        
        x2 = erfinv(x2)
        
        x2 = pd.DataFrame(x2)[0].replace(np.inf, 1).values
        x2 = pd.DataFrame(x2)[0].replace(-np.inf, -1).values
        
        data[col] = x2
    print('End RankGauss...')
    return data

In [279]:
X = RankGauss(temp_X)

Start RankGauss...
End RankGauss...


In [284]:
X = X.values

In [320]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
#     x = Dense(4096, activation='relu')(inputs) # 1500 original
#     x = Dropout(0.2)(x)
#     #x = Dense(2048, activation='relu')(x)
    
#     x = Dense(1500, activation='relu', name="feature")(x) # 1500 original
#     x = Dropout(0.15)(x)
#     #x = Dense(512, activation='relu')(x)
    #x = BatchNormalization()(inputs)
    x = Dense(256, activation='relu')(inputs) # 1500 original
    x = Dropout(0.2)(x)
    
#     x = BatchNormalization()(x)
#     x = Dense(64, activation='relu')(x) # 1500 original
#     x = Dropout(0.1)(x)
    #x = BatchNormalization()(x)
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [250]:
categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=6)

In [251]:
len(label)

116624

In [285]:
train_FE = X[:len(label)]

test_FE = X[len(label):]

In [158]:
index = pd.read_csv('index.csv')

In [ ]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 4s 47us/step - loss: 0.1686 - val_loss: 0.1099
Epoch 2/300
93297/93297 [==============================] - 5s 49us/step - loss: 0.1042 - val_loss: 0.0831
Epoch 3/300
93297/93297 [==============================] - 4s 46us/step - loss: 0.0843 - val_loss: 0.0707
Epoch 4/300
93297/93297 [==============================] - 4s 47us/step - loss: 0.0738 - val_loss: 0.0632
Epoch 5/300
93297/93297 [==============================] - 4s 47us/step - loss: 0.0673 - val_loss: 0.0582
Epoch 6/300
93297/93297 [==============================] - 4s 47us/step - loss: 0.0617 - val_loss: 0.0546
Epoch 7/300
93297/93297 [==============================] - 4s 47us/step - loss: 0.0568 - val_loss: 0.0508
Epoch 8/300
93297/93297 [==============================] - 4s 46us/step - loss: 0.0541 - val_loss: 0.0489
Epoch 9/300
93297/93297 [==============================] - 5s 48us/step - loss: 0.0513

In [324]:
meta_test

,0,1,2,3,4,5,0,1,2,3,...,2,3,4,5,0,1,2,3,4,5
0,0.999992,7.191005e-08,3.537207e-06,1.597396e-07,1.226421e-06,3.085586e-06,0.999957,5.943384e-07,5.980198e-07,8.033251e-07,...,8.301787e-07,1.514011e-08,2.268478e-08,8.591879e-07,0.999993,5.943508e-08,1.760353e-06,3.001410e-07,4.449218e-07,4.251663e-06
1,0.985951,3.039015e-03,3.610424e-04,2.187542e-03,4.202771e-04,8.040775e-03,0.983542,2.451134e-03,8.970613e-04,1.735783e-03,...,2.346894e-04,8.194821e-04,1.256705e-04,6.957083e-03,0.983052,4.363536e-04,4.623631e-04,2.126883e-03,1.509779e-04,1.377180e-02
2,0.999989,4.906688e-07,3.189032e-06,5.522800e-06,9.695473e-07,4.394901e-07,0.999966,5.354470e-06,9.620514e-06,1.605548e-05,...,4.755891e-06,2.635490e-06,2.269372e-06,3.312975e-07,0.999975,7.052127e-07,1.482779e-05,5.734949e-06,2.977273e-06,7.085253e-07
3,0.998606,8.622432e-05,4.130341e-04,1.512516e-04,1.277373e-04,6.157184e-04,0.998535,1.086043e-04,6.051570e-04,2.306225e-04,...,1.978367e-04,1.715648e-05,1.677516e-05,2.600068e-04,0.998382,8.860238e-05,5.183002e-04,9.249778e-05,9.988355e-05,8.183689e-04
4,0.999996,1.440747e-07,3.426323e-06,7.004025e-08,1.643256e-07,1.895784e-07,0.999994,1.933789e-07,5.496272e-06,1.298732e-07,...,5.844895e-06,1.369021e-07,1.679633e-07,3.439854e-07,0.999996,4.032757e-08,4.115305e-06,1.380727e-08,4.765591e-08,2.358917e-07
5,0.999798,5.995209e-06,6.046112e-05,1.152820e-06,4.357512e-06,1.297389e-04,0.999871,4.147463e-06,9.350754e-05,1.512484e-06,...,3.808624e-05,2.187237e-06,3.711645e-06,3.153190e-05,0.999849,2.556997e-06,9.872561e-05,1.321202e-06,1.275765e-06,4.738393e-05
6,0.998497,6.047039e-05,1.297129e-03,8.383538e-05,3.064692e-05,3.151812e-05,0.999129,5.149538e-05,6.918266e-04,8.254885e-05,...,1.592021e-04,7.151824e-06,9.418763e-06,2.218132e-06,0.999666,3.578079e-05,2.453306e-04,2.836065e-05,2.025484e-05,4.279005e-06
7,1.000000,1.030492e-07,1.952236e-07,2.296131e-08,7.310184e-08,7.492778e-09,1.000000,1.308903e-07,3.277795e-07,1.295933e-08,...,2.864797e-07,7.212320e-09,4.098437e-08,2.972338e-08,0.999999,2.440847e-07,3.797350e-07,9.575933e-09,2.231300e-08,5.852381e-08
8,0.999899,7.985399e-06,7.901366e-08,3.816459e-06,2.018627e-06,8.687787e-05,0.999916,5.192092e-06,2.920313e-07,8.342151e-07,...,1.250402e-08,5.877823e-07,3.021921e-07,1.633898e-05,0.999976,2.326444e-07,1.297926e-08,3.168728e-07,3.040934e-07,2.279271e-05
9,0.993838,2.071012e-04,5.149256e-03,2.563498e-04,2.394603e-04,3.096344e-04,0.989680,4.485562e-04,3.933638e-03,6.700848e-04,...,1.470410e-03,8.981874e-05,5.856593e-04,5.337076e-04,0.996890,1.957798e-04,2.018560e-03,8.131640e-05,4.457053e-04,3.683107e-04


In [322]:
pred

,file_id,prob0,prob1,prob2,prob3,prob4,prob5
0,0,0.999999,1.777955e-08,2.088523e-07,7.716872e-09,1.399030e-08,1.200984e-06
1,1,0.977021,6.130227e-03,1.727615e-03,3.683929e-03,7.134216e-04,1.072397e-02
2,2,0.999999,1.834109e-07,5.478809e-07,4.177589e-07,9.464071e-08,2.365052e-08
3,3,0.999578,7.255517e-05,5.439030e-05,9.640405e-05,4.780729e-05,1.506524e-04
4,4,0.999988,1.030776e-06,7.377798e-06,7.189824e-07,8.708814e-07,1.723857e-06
5,5,0.999590,3.687168e-05,2.450423e-04,2.048310e-05,1.114523e-05,9.616444e-05
6,6,0.999666,5.035452e-05,2.099675e-04,3.752954e-05,2.071647e-05,1.540020e-05
7,7,1.000000,9.955257e-08,1.296059e-07,2.480840e-08,2.823589e-08,6.732793e-09
8,8,0.999381,5.063404e-06,9.083445e-07,1.714678e-06,1.653603e-06,6.096920e-04
9,9,0.996974,2.832199e-04,1.003988e-03,2.431828e-04,1.002234e-04,1.394889e-03


In [325]:
pred = pd.DataFrame(sub_sample.file_id)

pred['prob0'] = meta_test[0].mean(axis = 1)
pred['prob1'] = meta_test[1].mean(axis = 1)
pred['prob2'] = meta_test[2].mean(axis = 1)
pred['prob3'] = meta_test[3].mean(axis = 1)
pred['prob4'] = meta_test[4].mean(axis = 1)
pred['prob5'] = meta_test[5].mean(axis = 1)

In [326]:
pred.to_csv('/home/libo/Security/sub/200+特征NN2.csv', index = None ,encoding = 'utf-8')

In [327]:
pred.mean()

file_id    26545.550781
prob0          0.905213
prob1          0.007373
prob2          0.012306
prob3          0.012010
prob4          0.001116
prob5          0.062060
dtype: float32

In [308]:
pred.mean()

file_id    26545.550781
prob0          0.905279
prob1          0.007626
prob2          0.012319
prob3          0.012203
prob4          0.001240
prob5          0.061403
dtype: float32

In [349]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
    x1 = BatchNormalization()(inputs)
    x1 = Dense(128, activation='relu')(x1) 
    x1 = Dropout(0.1)(x1)
    
    x2 = BatchNormalization()(inputs)
    x2 = Dense(128, activation='sigmoid')(x2) 
    x2 = Dropout(0.1)(x2)
    
    x3 = BatchNormalization()(inputs)
    x3 = Dense(128, activation='tanh')(x3) 
    x3 = Dropout(0.1)(x3)
    
    x = concatenate([x1, x2, x3])
    
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x) 
    x = Dropout(0.2)(x)
    
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [351]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 64
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 20s 213us/step - loss: 0.1024 - val_loss: 0.0514
Epoch 2/300
93297/93297 [==============================] - 19s 206us/step - loss: 0.0494 - val_loss: 0.0417
Epoch 3/300
93297/93297 [==============================] - 19s 208us/step - loss: 0.0403 - val_loss: 0.0366
Epoch 4/300
93297/93297 [==============================] - 19s 207us/step - loss: 0.0357 - val_loss: 0.0340
Epoch 5/300
93297/93297 [==============================] - 20s 209us/step - loss: 0.0330 - val_loss: 0.0325
Epoch 6/300
93297/93297 [==============================] - 19s 207us/step - loss: 0.0310 - val_loss: 0.0323
Epoch 7/300
93297/93297 [==============================] - 19s 208us/step - loss: 0.0278 - val_loss: 0.0295
Epoch 8/300
93297/93297 [==============================] - 19s 206us/step - loss: 0.0271 - val_loss: 0.0292
Epoch 9/300
93297/93297 [==============================] - 20s 210us/s

In [352]:
pred = pd.DataFrame(sub_sample.file_id)

pred['prob0'] = meta_test[0].mean(axis = 1)
pred['prob1'] = meta_test[1].mean(axis = 1)
pred['prob2'] = meta_test[2].mean(axis = 1)
pred['prob3'] = meta_test[3].mean(axis = 1)
pred['prob4'] = meta_test[4].mean(axis = 1)
pred['prob5'] = meta_test[5].mean(axis = 1)

In [353]:
pred.to_csv('/home/libo/Security/sub/200+特征NN3.csv', index = None ,encoding = 'utf-8')

In [354]:
pred.mean()

file_id    26545.550781
prob0          0.902574
prob1          0.007754
prob2          0.013352
prob3          0.013089
prob4          0.001145
prob5          0.062158
dtype: float32

In [136]:
def SwapNoise(X, p = 0.1):
    
    print('Start SwapNoise...')
    data = X.copy()
    n_sample = X.shape[0]
    
    for i in range(len(X.columns)):
        if i%100 == 0:
            print(i)
        feature = X.columns[i]

        selected_index_1 = random.sample(range(n_sample), int(p*n_sample))
        selected_index_2 = random.sample(range(n_sample), int(p*n_sample))

        new_values = X[feature].values[selected_index_2]
        
        data.loc[selected_index_1, feature] = new_values
    
    print('End SwapNoise...')
    
    return data

In [11]:
data_FE.fillna(0, inplace = True)

In [14]:
gc.collect()

0

In [17]:
100%100

0

In [130]:
def RankGauss(data):
    new_data = data.copy()
    print('Start RankGauss...')
    for i in range(len(data.columns)):
        if i%100 == 0:
            print(i)
        col = data.columns[i]
        #print('RankGauss...%s'%col)
        x = data[col].rank(method = 'dense')
        max_ = x.max()
        min_ = x.min()
        mean_ = (max_ + min_)/2
        
        x2 = (x - mean_).values/(0.0001+max_ - mean_)
        
        x2 = erfinv(x2)
        
        x2 = pd.DataFrame(x2)[0].replace(np.inf, 1).values
        x2 = pd.DataFrame(x2)[0].replace(-np.inf, -1).values
        
        new_data[col] = x2
    print('End RankGauss...')
    return new_data

In [170]:
data_FE

,file_id,file_id_sum,index_max,index_mean,index_median,index_std,index_skew,tid_unique_num,tid_max,tid_min,...,index_4991,index_4992,index_4993,index_4994,index_4995,index_4996,index_4997,index_4998,index_4999,index_5000
0,-4.301437,-1.411583,-0.965880,-0.957965,-0.971188,-0.622574,-4.131797,-3.553118,-1.029245,-0.699163,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
1,-3.098201,-0.850455,-0.582884,-1.528779,-2.502750,-1.923625,1.275020,-3.553118,-2.531207,-2.380000,...,-3.294543,-3.294543,-3.294543,-3.294543,-1.518551,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
2,-2.989700,-1.803785,-1.748369,-0.837557,-1.751835,-0.769413,-4.131797,-3.553118,-1.790164,-1.582936,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
3,-2.924533,-0.727215,-0.365032,-0.470137,-0.576386,-0.264127,1.037339,-1.642972,-0.463058,-3.801159,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
4,-2.877485,-0.258274,3.880604,-1.351327,-1.623813,-2.076036,0.712980,-3.553118,0.712871,-2.156595,...,-1.518551,-1.518551,-3.294543,-1.518551,-1.518551,-1.518551,-1.518551,-3.294543,-1.518551,-1.514028
5,-2.840506,-1.887625,-0.437417,-0.389999,0.559704,-0.302891,-4.131797,-3.553118,-0.529769,-0.022658,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
6,-0.328328,-2.103633,-0.641042,-2.089821,-1.052236,-0.654853,-1.399047,-1.983135,-1.674380,-0.442886,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
7,-2.783910,-2.000553,-1.807811,4.133740,-1.863040,-0.300360,0.386838,-1.819533,-1.923052,-2.004849,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
8,0.859206,-1.747585,-0.676061,-1.183206,-1.863040,-1.535171,0.000567,-1.983135,-1.599036,-1.443526,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213
9,-0.709614,-1.944193,-0.984529,-0.343786,-1.618889,-1.142092,0.981786,-3.553118,-1.659654,-1.438135,...,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.294543,-3.292213


In [174]:
%%time
X = RankGauss(data_FE[t])

Start RankGauss...
0
100
200
300
400
500
600
700
800
900
End RankGauss...
CPU times: user 46.2 s, sys: 2.13 s, total: 48.3 s
Wall time: 47 s


In [179]:
def get_DAE_output(X, feature_name, dim):
    
    X = X.fillna(0)
    
    try:
        X = X.drop('file_id', axis = 1)
    except:
        X = X
        
    #X = RankGauss(X)
#     mm = MinMaxScaler()
#     X = mm.fit_transform(X)
    X = pd.DataFrame(X)
    X_noise = SwapNoise(X, p = 0.1)

    def get_AE():
        # denoising autoencoder
        inputs = Input((X.shape[1],))

        x = Dense(12000, activation='relu')(inputs) # 1500 original
        #x = Dropout(0.1)(x)
        
        x = Dense(dim, activation='relu', name="feature")(x) # 1500 original

        x = Dense(12000, activation='relu')(x) # 1500 original
        #x = GaussianDropout(0.1)(x)

        outputs = Dense(X.shape[1], activation='linear')(x)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    ae = get_AE()
    ae.compile(optimizer='sgd', loss='mse')
    
    ae.fit(X_noise, X,epochs=30,
                batch_size=128,
                shuffle=True)
    
    get_dae_output = Model(inputs=ae.input,
                                     outputs=ae.get_layer('feature').output)
    dea_feature = get_dae_output.predict(X_noise)
    dea_feature = pd.DataFrame(dea_feature)
    dea_feature.columns = ['dae_' + feature_name + str(x) for x in range(dim)]

    return dea_feature

In [180]:
dea_feature = get_DAE_output(X, 'feature_top1000_', 3000)
dea_feature.to_pickle('dae_feature_top1000_.pkl')

Start SwapNoise...
0
100
200
300
400
500
600
700
800
900
End SwapNoise...
Epoch 1/30
169717/169717 [==============================] - 858s 5ms/step - loss: 0.7427
Epoch 2/30
169717/169717 [==============================] - 859s 5ms/step - loss: 0.5666
Epoch 3/30
169717/169717 [==============================] - 859s 5ms/step - loss: 0.5000
Epoch 4/30
169717/169717 [==============================] - 860s 5ms/step - loss: 0.4541
Epoch 5/30
169717/169717 [==============================] - 862s 5ms/step - loss: 0.4199
Epoch 6/30
169717/169717 [==============================] - 860s 5ms/step - loss: 0.3934
Epoch 7/30
169717/169717 [==============================] - 858s 5ms/step - loss: 0.3721
Epoch 8/30
169717/169717 [==============================] - 861s 5ms/step - loss: 0.3546
Epoch 9/30
169717/169717 [==============================] - 863s 5ms/step - loss: 0.3399
Epoch 10/30
169717/169717 [==============================] - 859s 5ms/step - loss: 0.3273
Epoch 11/30
169717/169717 [========

In [181]:
dea_feature

,dae_feature_top1000_0,dae_feature_top1000_1,dae_feature_top1000_2,dae_feature_top1000_3,dae_feature_top1000_4,dae_feature_top1000_5,dae_feature_top1000_6,dae_feature_top1000_7,dae_feature_top1000_8,dae_feature_top1000_9,...,dae_feature_top1000_2990,dae_feature_top1000_2991,dae_feature_top1000_2992,dae_feature_top1000_2993,dae_feature_top1000_2994,dae_feature_top1000_2995,dae_feature_top1000_2996,dae_feature_top1000_2997,dae_feature_top1000_2998,dae_feature_top1000_2999
0,1.881512,2.920863,1.487667,4.117405,0.000000,2.232470,0.840019,2.697116,1.278547,2.550312,...,0.0,2.340072,2.020980,2.176947,1.928013,1.756413,0.0,0.906656,1.097359,2.958768
1,2.632660,1.058027,1.496170,5.736566,0.000000,1.936733,0.671353,4.017149,1.569130,3.259978,...,0.0,3.079387,2.931926,1.707490,2.116532,1.905017,0.0,1.553054,1.384140,2.826353
2,1.958369,2.730667,1.698328,5.176198,0.000000,2.852864,1.300755,2.494637,0.611089,3.153272,...,0.0,2.477732,2.473279,1.754416,2.163231,1.636379,0.0,1.431715,1.314316,3.013905
3,3.479455,3.676176,3.219994,4.484783,0.178353,2.868507,2.608946,3.277121,1.529822,2.688811,...,0.0,2.760759,1.922472,2.059581,3.075660,2.316586,0.0,1.655438,2.086109,2.581993
4,3.364782,2.967819,2.286239,4.432095,0.000000,2.132872,2.946206,3.124769,1.237509,1.821666,...,0.0,2.952417,0.998453,2.390120,1.906370,2.996024,0.0,1.528528,1.500893,2.866356
5,3.034280,2.369850,1.699591,3.851523,0.000000,2.260962,0.895022,2.775290,1.983313,1.437356,...,0.0,2.404657,2.325013,2.083134,3.082900,1.288329,0.0,1.178575,0.638367,2.284507
6,2.496743,0.805388,2.713860,3.256122,0.000000,3.138455,2.070596,2.694432,2.864106,2.736871,...,0.0,2.280457,1.233942,1.813007,2.706137,2.277993,0.0,0.919230,1.773854,2.191049
7,1.119019,1.216427,2.408917,4.144304,0.000000,2.138282,1.008365,3.104560,1.729284,2.659860,...,0.0,3.081520,2.159670,0.468791,2.225439,1.502474,0.0,0.766038,1.195077,2.022107
8,2.003590,1.640047,1.020384,4.990364,0.000000,2.013547,0.837889,2.340847,1.905562,1.833134,...,0.0,3.368729,2.177452,1.098775,3.008877,1.579340,0.0,1.340394,1.521882,2.799715
9,2.043662,1.907874,1.715605,4.974628,0.000000,2.517453,0.612909,3.069865,1.246382,3.380193,...,0.0,2.719014,2.426549,1.706939,1.996956,1.682201,0.0,1.341147,1.640485,3.195219


In [214]:
X = RankGauss(dea_feature)

Start RankGauss...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
End RankGauss...


In [202]:
# mm = MinMaxScaler()
# X = mm.fit_transform(dea_feature)

In [218]:
X = X.values

In [221]:
mm = MinMaxScaler()
X2 = mm.fit_transform(dea_feature)

In [222]:
train_FE = X2[:len(label)]

test_FE = X2[len(label):]

In [204]:
categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=6)

In [223]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
#     x = Dense(4096, activation='relu')(inputs) # 1500 original
#     x = Dropout(0.2)(x)
#     #x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(inputs)
    x = Dense(1024, activation='relu', name="feature")(x) # 1500 original
    x = Dropout(0.15)(x)
    
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    x = BatchNormalization()(x)
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [ ]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 42s 451us/step - loss: 0.3527 - val_loss: 0.0820
Epoch 2/300
93297/93297 [==============================] - 40s 431us/step - loss: 0.0869 - val_loss: 0.0642
Epoch 3/300
93297/93297 [==============================] - 41s 440us/step - loss: 0.0680 - val_loss: 0.0557
Epoch 4/300
93297/93297 [==============================] - 40s 430us/step - loss: 0.0584 - val_loss: 0.0512
Epoch 5/300
93297/93297 [==============================] - 40s 434us/step - loss: 0.0533 - val_loss: 0.0482
Epoch 6/300
93297/93297 [==============================] - 40s 431us/step - loss: 0.0477 - val_loss: 0.0454
Epoch 7/300
93297/93297 [==============================] - 41s 438us/step - loss: 0.0433 - val_loss: 0.0433
Epoch 8/300
93297/93297 [==============================] - 42s 447us/step - loss: 0.0392 - val_loss: 0.0425
Epoch 9/300
93297/93297 [==============================] - 41s 435us/s

In [225]:
meta_test

,0,1,2,3,4,5,0,1,2,3,...,2,3,4,5,0,1,2,3,4,5
0,9.936101e-01,0.000123,2.600843e-04,1.251634e-04,3.030219e-04,5.578864e-03,9.988226e-01,1.081241e-04,9.346836e-05,5.522501e-04,...,3.468884e-06,3.971810e-05,3.185242e-05,2.848149e-02,0.999693,1.443596e-05,6.681529e-06,8.207291e-06,3.200722e-06,2.748330e-04
1,9.982349e-01,0.000424,2.951183e-04,2.781055e-04,1.490421e-04,6.187411e-04,9.941604e-01,1.100011e-03,1.581753e-03,5.972882e-04,...,2.218895e-03,1.766880e-04,1.578653e-04,2.536919e-04,0.995486,1.429911e-04,8.219851e-04,1.149163e-03,4.093489e-04,1.990821e-03
2,9.996393e-01,0.000090,4.463020e-05,1.209335e-04,6.116525e-05,4.346172e-05,9.998193e-01,5.246400e-05,4.614759e-05,3.588556e-05,...,5.178270e-05,8.863331e-05,6.915889e-05,1.069107e-04,0.999838,6.048466e-05,1.760708e-05,2.530723e-05,5.029561e-05,8.532630e-06
3,9.992447e-01,0.000065,3.007085e-05,6.321436e-05,9.665532e-05,5.007330e-04,9.996965e-01,2.459122e-05,3.222423e-05,4.674298e-05,...,1.454176e-04,2.438268e-04,7.789575e-05,6.699206e-04,0.999244,2.694070e-05,7.841168e-05,2.321944e-05,7.106430e-05,5.562644e-04
4,9.997780e-01,0.000041,8.963489e-05,3.893605e-05,3.172006e-05,2.123621e-05,9.987514e-01,2.241151e-04,5.265015e-04,1.547950e-04,...,3.785608e-05,6.522154e-06,1.199859e-05,8.210759e-06,0.999911,9.947992e-06,2.072154e-05,1.794418e-05,2.055996e-05,1.954629e-05
5,9.999251e-01,0.000026,1.119981e-05,1.419489e-05,1.432378e-05,9.028975e-06,9.998952e-01,1.477781e-05,3.003020e-05,2.654633e-05,...,2.974226e-05,2.543379e-05,1.647904e-05,6.666316e-06,0.999793,1.004206e-04,3.761602e-05,2.714118e-05,2.411828e-05,1.725524e-05
6,9.988237e-01,0.000213,1.101635e-04,1.162126e-04,2.128626e-04,5.235389e-04,9.994752e-01,4.347370e-05,9.059020e-05,1.140496e-04,...,3.528762e-04,1.485100e-04,1.805617e-04,1.642209e-04,0.999894,5.385070e-06,1.279028e-05,6.679069e-06,1.014137e-05,7.126426e-05
7,9.999735e-01,0.000003,4.701211e-06,1.055087e-05,7.184451e-06,8.267497e-07,9.999601e-01,1.186653e-05,3.919249e-06,1.395075e-05,...,3.880146e-07,9.248472e-07,6.609907e-07,1.026621e-06,0.999995,6.756937e-07,9.915142e-07,5.687025e-07,2.009401e-06,4.139313e-07
8,9.999251e-01,0.000047,4.416764e-06,4.181895e-06,4.200361e-06,1.504574e-05,9.999825e-01,2.080137e-06,4.842223e-06,4.771045e-06,...,4.591039e-06,2.903263e-06,2.242103e-06,3.488610e-06,0.999986,8.175695e-07,2.059060e-06,5.798682e-06,2.339624e-06,2.682057e-06
9,9.979213e-01,0.000245,6.253995e-04,6.296746e-04,3.769306e-04,2.015510e-04,9.990975e-01,4.424142e-05,2.713389e-04,2.867712e-04,...,1.128900e-03,1.940055e-03,9.765518e-04,7.473261e-05,0.996626,2.134753e-04,5.600248e-04,1.828161e-03,6.506039e-04,1.221330e-04


In [228]:
pred = pd.DataFrame(sub_sample.file_id)

pred['prob0'] = meta_test[0].mean(axis = 1)
pred['prob1'] = meta_test[1].mean(axis = 1)
pred['prob2'] = meta_test[2].mean(axis = 1)
pred['prob3'] = meta_test[3].mean(axis = 1)
pred['prob4'] = meta_test[4].mean(axis = 1)
pred['prob5'] = meta_test[5].mean(axis = 1)

pred.to_csv('/home/libo/Security/sub/8_15_DAE_v1.csv', index = None ,encoding = 'utf-8')

In [ ]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
#     x = Dense(4096, activation='relu')(inputs) # 1500 original
#     x = Dropout(0.2)(x)
#     #x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(inputs)
    x = Dense(512, activation='relu', name="feature")(x) # 1500 original
    x = Dropout(0.15)(x)
    
#     x = BatchNormalization()(x)
#     x = Dense(256, activation='relu')(x) # 1500 original
#     x = Dropout(0.1)(x)
    
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    x = BatchNormalization()(x)
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [ ]:
%%time
#logloss_list = []
meta_test2 = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v2_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test2 = pd.concat([meta_test2, pd.DataFrame(prob)], axis = 1)

In [226]:
meta_test2

NameError: name 'meta_test2' is not defined

In [ ]:
train_FE = X[:len(label)]

test_FE = X[len(label):]

In [ ]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
#     x = Dense(4096, activation='relu')(inputs) # 1500 original
#     x = Dropout(0.2)(x)
#     #x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(inputs)
    x = Dense(1024, activation='relu', name="feature")(x) # 1500 original
    x = Dropout(0.15)(x)
    
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    x = BatchNormalization()(x)
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [ ]:
%%time
#logloss_list = []
meta_test3 = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v3_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test3 = pd.concat([meta_test3, pd.DataFrame(prob)], axis = 1)

In [ ]:
def NN():
    # denoising autoencoder
    inputs= Input((train_FE.shape[1],))
    
#     x = Dense(4096, activation='relu')(inputs) # 1500 original
#     x = Dropout(0.2)(x)
#     #x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(inputs)
    x = Dense(512, activation='relu', name="feature")(x) # 1500 original
    x = Dropout(0.15)(x)
    
#     x = BatchNormalization()(x)
#     x = Dense(256, activation='relu')(x) # 1500 original
#     x = Dropout(0.1)(x)
    
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x) # 1500 original
    x = Dropout(0.1)(x)
    x = BatchNormalization()(x)
    outputs = Dense(6, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=[outputs])

    return model

In [ ]:
%%time
#logloss_list = []
meta_test4 = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v4_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test4 = pd.concat([meta_test4, pd.DataFrame(prob)], axis = 1)

In [ ]:
%%time
#logloss_list = []
meta_test5 = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 64
    bst_model_path = '5folds_' + 'nn_v5_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test5 = pd.concat([meta_test5, pd.DataFrame(prob)], axis = 1)

In [188]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 42s 446us/step - loss: 0.3578 - val_loss: 0.0772
Epoch 2/300
93297/93297 [==============================] - 40s 430us/step - loss: 0.0847 - val_loss: 0.0605
Epoch 3/300
93297/93297 [==============================] - 40s 430us/step - loss: 0.0672 - val_loss: 0.0537
Epoch 4/300
93297/93297 [==============================] - 40s 426us/step - loss: 0.0569 - val_loss: 0.0487
Epoch 5/300
93297/93297 [==============================] - 40s 424us/step - loss: 0.0506 - val_loss: 0.0459
Epoch 6/300
93297/93297 [==============================] - 39s 423us/step - loss: 0.0463 - val_loss: 0.0440
Epoch 7/300
93297/93297 [==============================] - 39s 422us/step - loss: 0.0409 - val_loss: 0.0432
Epoch 8/300
93297/93297 [==============================] - 39s 422us/step - loss: 0.0391 - val_loss: 0.0414
Epoch 9/300
93297/93297 [==============================] - 39s 421us/s

KeyboardInterrupt: 

In [186]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 16s 174us/step - loss: 0.3734 - val_loss: 0.0865
Epoch 2/300
93297/93297 [==============================] - 16s 176us/step - loss: 0.0879 - val_loss: 0.0667
Epoch 3/300
93297/93297 [==============================] - 16s 170us/step - loss: 0.0702 - val_loss: 0.0585
Epoch 4/300
93297/93297 [==============================] - 16s 170us/step - loss: 0.0611 - val_loss: 0.0539
Epoch 5/300
93297/93297 [==============================] - 16s 172us/step - loss: 0.0542 - val_loss: 0.0503
Epoch 6/300
93297/93297 [==============================] - 16s 174us/step - loss: 0.0491 - val_loss: 0.0483
Epoch 7/300
93297/93297 [==============================] - 17s 177us/step - loss: 0.0460 - val_loss: 0.0470
Epoch 8/300
93297/93297 [==============================] - 16s 173us/step - loss: 0.0427 - val_loss: 0.0450
Epoch 9/300
93297/93297 [==============================] - 16s 175us/s

KeyboardInterrupt: 

In [169]:
%%time
#logloss_list = []
meta_test = pd.DataFrame()
for i in range(5):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['file_id'])
    idx_val = np.array(index[index['fold'] == i]['file_id'])
    
    
    X_train = train_FE[idx_train]
    y_train = categorical_labels[idx_train]

    X_val = train_FE[idx_val]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    
    BATCH_SIZE = 128
    bst_model_path = '5folds_' + 'nn_v1_' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = NN()
    model.compile(optimizer='sgd', loss='categorical_crossentropy')
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=300, batch_size=BATCH_SIZE, shuffle=True, \
            callbacks=callbacks)
    
    model = NN()
    model.load_weights(bst_model_path)

    prob = model.predict(test_FE,batch_size=128,verbose=1)

    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 93297 samples, validate on 23327 samples
Epoch 1/300
93297/93297 [==============================] - 28s 295us/step - loss: 0.3931 - val_loss: 0.1019
Epoch 2/300
93297/93297 [==============================] - 27s 286us/step - loss: 0.1146 - val_loss: 0.0831
Epoch 3/300
93297/93297 [==============================] - 27s 285us/step - loss: 0.0973 - val_loss: 0.0830
Epoch 4/300
93297/93297 [==============================] - 27s 287us/step - loss: 0.0897 - val_loss: 0.0811
Epoch 5/300
93297/93297 [==============================] - 27s 290us/step - loss: 0.0817 - val_loss: 0.0702
Epoch 6/300
93297/93297 [==============================] - 27s 286us/step - loss: 0.0796 - val_loss: 0.0637
Epoch 7/300
93297/93297 [==============================] - 27s 293us/step - loss: 0.0754 - val_loss: 0.0654
Epoch 8/300
93297/93297 [==============================] - 27s 288us/step - loss: 0.0733 - val_loss: 0.0645
Epoch 9/300
93297/93297 [==============================] - 27s 288us/s

KeyboardInterrupt: 

In [208]:
top_feature = data_FE[t]

In [209]:
top_feature.to_pickle('top_feature1000.pkl')